## Workshop 1 Basic Python Code and Simple A/B Test on Landing Page

# Introduction

This workshop reviews the concepts of Python code syntax and gets students/participants to familiar with Jupyter Notebook setting for the data analytics been used later for A/B Test applications.

The case here is about the project in the course https://www.udacity.com/course/ab-testing--ud257 which design to experiment on the response to the landing page of an online learning course

The following libraries are imported to do the calculation (A typical Python programme makes use of other people open source codes to develop the algorithm for particular application)

Jupyter notebook is an interface platform for Python high level programme language that one can insert explanations and the codes, along the context as a document so that any readers can follow through the usage of the code and understand the underlying procedures. Jupyter notebook is very common for python data scientist for mutual reference.

In [ ]:
from scipy import stats
import pandas as pd

# Python libraries:

random, datetime for common language program function

pandas for data file or database manilpulation

seaborn for data visulation

statistics, numpy and scipy are for statistical and mathematical formula/function

In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import statsmodels

In [ ]:
import statsmodels.api as sm

In [ ]:
df = pd.read_csv("ab_data.csv")

In [ ]:
df.head()

# Explanatory Data Analysis 

check if there are miss alignments between c/t groups and landing pages design

In [ ]:
new_control = len(df[(df['landing_page']=='new_page') & (df['group']=='control')])

In [ ]:
new_control

In [ ]:
old_treatment =  len(df[(df['landing_page']=='old_page') & (df['group']=='treatment')])

In [ ]:
old_treatment

drop miss aligned rows

In [ ]:
df = df.drop(df.index[(df['landing_page']=='new_page') & (df['group']=='control')],axis = 0)

In [ ]:
df = df.drop(df.index[(df['landing_page']=='old_page') & (df['group']=='treatment')],axis = 0)

In [ ]:
df.shape

In [ ]:
df.isna().any()

check the conversion rates

In [ ]:
old_page_conversion = len(df[(df['landing_page']=='old_page') & (df['converted']==1)])/len(df[df['landing_page']=='old_page'])

In [ ]:
old_page_conversion

In [ ]:
new_page_conversion = len(df[(df['landing_page']=='new_page') & (df['converted']==1)])/len(df[df['landing_page']=='new_page'])

In [ ]:
new_page_conversion

check if there any duplicates

In [ ]:
len(df.user_id.unique())

create df2 with only first record of the duplicate user

In [ ]:
df[df.duplicated('user_id') == True]

In [ ]:
df[df['user_id'] == 773192]

In [ ]:
df2 = df.drop(index = 1899, axis = 0)

In [ ]:
df.shape

In [ ]:
df2.shape

The probability to receive treatment

In [ ]:
ct_rate = len(df[df['landing_page']=='new_page'])/len(df)

In [ ]:
ct_rate

In [ ]:
df2['landing_page'].value_counts().plot(kind='pie', figsize=(8,8));

In [ ]:
df2['day'] = df2['timestamp'].apply(lambda x:x.split(' ')[0])

In [ ]:
df2.head()

In [ ]:
df_cr = df2.groupby(['timestamp'])['converted'].apply(lambda g: g / g.sum()).rename("count")

In [ ]:
df_cr.head()

In [ ]:
df2

In [ ]:
df2['landing_page'].value_counts().plot(kind='pie', figsize=(8,8));

# plt.scatter(df2['day'],  df2['old_rate'], color='green',label='p_old')
# plt.scatter(df2['day'], df2['new_rate'], color='red',label='p_new')
# plt.xlabel('day of January')
# plt.ylabel('conversion rate')
# #plt.ylim(0.11, 0.13)
# plt.legend()
# plt.title('conversion rate at different days of the month');

# A/B Testing 

Null Hypothesis: Conversion(old) >= Conversion(new)

Alternative Hypothesis: Conversion(old) < Conversion(new)

Type I error. A Type I error occurs when the researcher rejects a null hypothesis when it is true. The probability of committing a Type I error is called the significance level. This probability is also called alpha, and is often denoted by α.

Type II error. A Type II error occurs when the researcher fails to reject a null hypothesis that is false. The probability of committing a Type II error is called Beta, and is often denoted by β. The probability of not committing a Type II error is called the Power of the test.

In [ ]:
n_new = len(df2[df2['landing_page']=='new_page'])
n_old = len(df2[df2['landing_page']=='old_page'])

In [ ]:
count_new = len(df2[(df2['landing_page']=='new_page') & (df2['converted']==1)])
nobs_new = len(df2[df2['landing_page']=='new_page'])
count_old = len(df2[(df2['landing_page']=='old_page') & (df2['converted']==1)])
nobs_old = len(df2[df2['landing_page']=='old_page'])

In [ ]:
stat, pval = sm.stats.proportions_ztest([count_new,count_old], [nobs_new,nobs_old], alternative='larger')

In [ ]:
pval

In [ ]:
from statsmodels.stats.power import TTestIndPower


In [ ]:
effect = 0.01
alpha = 0.05
power = 0.8
# perform power analysis
analysis = TTestIndPower()
result = analysis.solve_power(effect, power=power, nobs1=None, ratio=1.0, alpha=alpha)


In [ ]:
result

Since p-value = 0.905 >> alpha = 0.05, the nwe fail to reject null hypothesis, so that we will accept null hypothesis at 0.95 significance level. So the conversion rate of new version is not statistically significant larger than old version's conversion rate.